# Analyzing the Mojito Album Cover using `ssd`

> 麻烦给我的爱人来一杯 Mojito 🎵🎵🎵

This notebook demonstrates how to use Liga Vision to analyze the Mojito album cover.

## Image Visualization

In [1]:
from ligavision.spark.types import Image
uri = 'https://i.scdn.co/image/ab67616d0000b273466def3ce70d94dcacb13c8d'
image = Image(uri)
image * 0.5

Image(<embedded>)

## Initialization
+ Initialize the Spark Session with Rikai support
+ Register all built-in UDFs
+ Create the yolov5m model

In [2]:
from ligavision.spark import init_session

spark = init_session()

2023-02-17 11:30:22,963 INFO Rikai (__init__.py:127): setting spark.sql.extensions to net.xmacs.liga.spark.RikaiSparkSessionExtensions,org.apache.spark.sql.rikai.LigaImageExtensions
2023-02-17 11:30:22,964 INFO Rikai (__init__.py:127): setting spark.driver.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-02-17 11:30:22,964 INFO Rikai (__init__.py:127): setting spark.executor.extraJavaOptions to -Dio.netty.tryReflectionSetAccessible=true
2023-02-17 11:30:22,965 INFO Rikai (__init__.py:127): setting spark.jars to https://github.com/liga-ai/ligavision/releases/download/ligavision_0.2.0/liga-image-assembly_2.12-0.2.0.jar
23/02/17 11:30:23 WARN Utils: Your hostname, tubi resolves to a loopback address: 127.0.1.1; using 192.168.31.32 instead (on interface wlp0s20f3)
23/02/17 11:30:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To ad

In [4]:
model = "ssd"
spark.sql(f"""
CREATE MODEL {model}
FLAVOR pytorch
MODEL_TYPE {model}
""")

DataFrame[]

In [5]:
spark.sql(f"""
select pred.box, pred.label, pred.score
from (
  select explode(ML_PREDICT({model}, image('{uri}'))) as pred
)
""").toPandas()

/home/da/.cache/pants/named_caches/pex_root/venvs/d44d4dd571248e4aec9224fd2cac0d3ffa6602a1/c9e0fa5a0599b5c0fe562196d5f09e2a70435b71/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/da/.cache/pants/named_caches/pex_root/venvs/d44d4dd571248e4aec9224fd2cac0d3ffa6602a1/c9e0fa5a0599b5c0fe562196d5f09e2a70435b71/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=SSD300_VGG16_Weights.COCO_V1`. You can also use `weights=SSD300_VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


,box,label,score
0,"(261.00732421875, 184.5063934326172, 369.66735...",person,0.997334
1,"(270.88720703125, 210.1733856201172, 640.00006...",car,0.880633
2,"(307.9022216796875, 146.6266326904297, 448.873...",person,0.809895


## Q: How many cars/persons in this picture

In [6]:
df = spark.sql(f"""
select pred.label, count(*)
from (
  select *, explode(ML_PREDICT({model}, image)) as pred
  from (
    select image('{uri}') as image
  )
)
group by pred.label
""")
df.toPandas()

,label,count(1)
0,person,2
1,car,1


## Q2: Where are these cars and persons?

In [7]:
from ligavision.dsl import Text
from ligavision.dsl import Image
from functools import reduce

df = spark.sql(f"""
select * from (
  select explode(ML_PREDICT({model}, image)) as pred
  from (
    select image('{uri}') as image
  )
)
where pred.label = 'person' -- truck/car/person
""")
preds = [row.pred for row in df.collect()]

def add_layer(image, pred):
    return image | pred.box | Text(pred.label, (pred.box.xmin, pred.box.ymin - 10))

reduce(lambda a,b: add_layer(a, b), [image] + preds)

Draw(Box2d(xmin=261.00732421875, ymin=184.5063934326172, xmax=369.6673583984375, ymax=572.9732666015625))